In [126]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# prepare 
def split_sequence(sequence1, sequence2, n_steps):
    X, y = list(), list()
    for i in range(len(sequence1)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence1)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence1[i:end_ix]
        X.append(seq_x)
        y.append(sequence2[end_ix])
    return np.array(X), np.array(y)

In [ ]:
# set hyperparameters

In [ ]:
cur_master = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/battery/Battery_Current.csv', delimiter=';')
cur_master = cur_master[0:150000,1]

volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_slave_0_cell_4 = volt_slave_0_cell_4[0:150000, 4]

n_steps = 10
n_features = 1

X_train, y_train = split_sequence(cur_master, volt_slave_0_cell_4, n_steps)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], n_features)
X_train.shape

In [ ]:
model = keras.Sequential()

#Adding the first LSTM layer and some Dropout regularisation
model.add(layers.LSTM(units = 50, activation='relu', return_sequences = True, input_shape = (n_steps, n_features)))

# Adding the output layer
model.add(layers.Dense(units = 1))

# Show model
model.summary()

In [ ]:
# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 10, verbose = 1)

In [ ]:
test_cur_master = np.loadtxt('../data/fobss_data/data/osc_19_11_18/battery/Battery_Current.csv', delimiter=';')
test_cur_master = test_cur_master[1:,1]

test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/osc_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:, 4]

X_test, y_test = split_sequence(test_cur_master, test_volt_slave_0_cell_4, n_steps)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], n_features)
X_test.shape

In [ ]:
yhat = model.predict(X_test, verbose = 1)

plt.plot(yhat[:,0,0], color='red', label = 'predicted')
plt.plot(y_test, color='blue', label = 'measured')
plt.legend()
plt.show()